# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as L
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor, StochasticWeightAveraging

import app.training.networks as net
from app.training.dataset.data_module import IVS_DataModule

# NN1 Residual Feed Forward NN

In [2]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 17604), started 10:34:12 ago. (Use '!kill 17604' to kill it.)

In [ ]:
DataLoader = IVS_DataModule()

r,q,v0,kappa,theta,sigma,rho,lambda_p,tau_1,tau_2,tau_3,tau_4,tau_5,tau_6,tau_7,tau_8,tau_9,tau_10,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,m_13,nu_1,nu_2,nu_3,nu_4,nu_5,delta_1,delta_2,delta_3,delta_4,delta_5,implied_vol_surface
f32,f32,f32,f32,f32,f32,f32,f32,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,list[f32]
0.042819,0.02408,0.075004,9.563508,0.234728,1.410849,-0.050167,0.221455,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,-0.06145,-0.261182,0.014085,0.040484,0.226299,0.231427,0.243308,0.203859,0.250898,0.209558,"[0.551366, 0.474349, … 0.488375]"
0.047914,0.021959,0.059782,6.615184,0.338144,0.527069,-0.122137,0.218679,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,0.251862,-0.084615,0.099564,0.127918,-0.013492,0.223936,0.205652,0.203066,0.264801,0.205546,"[0.703693, 0.609873, … 0.566131]"
0.04352,0.00934,0.045393,9.307867,0.145592,1.447328,0.018445,0.448545,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,0.158138,0.203891,-0.026683,-0.095697,0.107908,0.251708,0.269171,0.277242,0.204776,0.227962,"[0.738972, 0.645014, … 0.399378]"
0.034193,0.012755,0.082381,8.083626,0.12598,0.529315,0.250438,0.364253,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,0.186329,-0.143617,0.104117,-0.20423,0.255101,0.272728,0.297045,0.237253,0.258477,0.240227,"[0.74685, 0.649684, … 0.404359]"
0.041095,0.028444,0.066657,9.0394,0.292203,0.423733,-0.001932,0.351646,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,0.167266,-0.21599,0.132366,-0.011974,0.124185,0.247581,0.278154,0.215618,0.247202,0.211566,"[0.716889, 0.623773, … 0.538575]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.021236,0.010544,0.08991,6.869073,0.332751,1.178937,-0.010981,0.277244,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,0.061865,-0.014683,0.000728,0.094775,0.155623,0.277628,0.208675,0.27168,0.278489,0.218587,"[0.656443, 0.5669, … 0.567817]"
0.014792,0.00726,0.068549,6.734044,0.119505,0.533739,-0.020346,0.178137,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,-0.050293,-0.278203,0.230263,-0.001027,-0.09365,0.218623,0.214988,0.264861,0.273286,0.2186,"[0.53584, 0.460477, … 0.346909]"
0.039311,0.01098,0.101494,6.134017,0.230201,1.154013,-0.220341,0.227497,7,11,17,27,44,69,110,175,277,440,0.8,0.833333,0.866667,0.9,0.933333,0.966667,1.0,1.033333,1.066667,1.1,1.133333,1.166667,1.2,-0.083854,0.019652,-0.077777,-0.280855,-0.286169,0.209684,0.207535,0.281355,0.289655,0.292383,"[0.524997, 0.452119, … 0.4888]"


In [4]:
nn1 = net.NN1Residual()

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    save_top_k=1,
    mode="min",
    filename="best_model-{epoch:02d}-{val_loss:.2f}",
    save_weights_only=True
)

early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min",
    verbose=True
)

stochastic_weight_avg = StochasticWeightAveraging(1e-3)

lr_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = L.Trainer(accelerator='gpu', 
                    callbacks=[early_stopping_callback, checkpoint_callback, lr_monitor, stochastic_weight_avg], 
                    max_epochs=20, 
                    deterministic=True)

trainer.fit(nn1, DataLoader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type       | Params | Mode 
-------------------------------------------------------------
0 | mse_loss              | MSELoss    | 0      | train
1 | r2Score               | R2Score    | 0      | train
2 | activation            | ELU        | 0      | train
3 | input_head            | Sequential | 43.0 K | train
4 | hidden_residual_block | Sequential | 4.2 M  | train
5 | output_layer          | Linear     | 133 K  | train
-------------------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.499    Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\USUARIO\PycharmProjects\MathThesis\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\USUARIO\PycharmProjects\MathThesis\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [11]:
trainer.test(nn1, DataLoader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\USUARIO\PycharmProjects\MathThesis\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 adj_test_r2_score_epoch    0.42912957072257996
        test_loss          0.0020858978386968374
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0020858978386968374,
  'adj_test_r2_score_epoch': 0.42912957072257996}]

In [ ]:
nn2 = net.NN2()

dummy_input = torch.randn(1, 133)
nn2.eval()
output = nn2(dummy_input)
print("Output shape:", output.shape)  # Should be (1, 10)
assert output.shape == (1, 10)
print(nn2)

In [ ]:
# Instantiate networks
nn1 = net.NN1Residual()
nn2 = net.NN2()
nn3 = net.NN3(nn1, nn2)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(nn3.parameters(), lr=0.001)

In [ ]:
# Training loop for NN3
def train_nn3(nn3, optimizer, criterion, train_params, target_params, epochs=50, batch_size=64):
    num_batches = len(train_params) // batch_size
    for epoch in range(epochs):
        total_loss = 0.0
        for i in range(num_batches):
            # Extract batch data
            batch_params = train_params[i * batch_size:(i + 1) * batch_size]
            batch_target = target_params[i * batch_size:(i + 1) * batch_size]

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = nn3(batch_params)
            loss = criterion(outputs, batch_target)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / num_batches:.4f}")



# Train the network
train_nn3(nn3, optimizer, criterion, train_params, train_params, epochs=50)


In [ ]:
from sklearn.metrics import accuracy_score


def test_nn3(nn3, test_params, test_target):
    nn3.eval()  # Set to evaluation mode
    with torch.no_grad():
        predictions = nn3(test_params)
        loss = criterion(predictions, test_target)
    print(f"Test Loss: {loss.item():.4f}")
    # print(f"Accuracy: {accuracy_score(predictions, test_target):.4f}")

# Test the network
test_nn3(nn3, test_params, test_params)